## Litter Windrow dataset reconstruction
not working yet, there is a bug in the mask reconstruction,
doesn't overlap the image and mask yet,
provides a good overview of the needed functionality

In [1]:
!pip -q install gdown sentinelhub cartopy

In [14]:
import gdown
import os
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from PIL import Image
from datetime import datetime


from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

import geopandas as gpd
from shapely.geometry import box
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import box
import cartopy

config = SHConfig()
config.sh_client_id = "sh-84d21c38-65f5-410d-9267-b1db4ba47b41"
config.sh_client_secret = "7ykprUj53bYD42W30gevmHFnI4U5y75M"
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

catalog = SentinelHubCatalog(config=config)
resolution = 10

In [3]:
url = "https://zenodo.org/records/11045944/files/WASP_LW_SENT2_MED_L1C_B_201506_202109_10m_6y_NRT_v1.0.nc?download=1"
gdown.download(url, "data.nc")

Downloading...
From: https://zenodo.org/records/11045944/files/WASP_LW_SENT2_MED_L1C_B_201506_202109_10m_6y_NRT_v1.0.nc?download=1
To: /content/data.nc
100%|██████████| 2.06G/2.06G [01:16<00:00, 27.0MB/s]


'data.nc'

In [4]:
!ls

data.nc  Images  LW_centroids.xlsx  sample_data


In [15]:
###################### Mars Sara's Code ######################
def download_sentinel_images(aoi_coords_wgs84, time_interval, output_dir="Images", resolution=10, max_size=2500):
    """
    Downloads Sentinel-2 true color images for a given AOI and time range.

    Parameters:
    - aoi_coords_wgs84: list of [min_lon, min_lat, max_lon, max_lat] (AOI bounding box)
    - time_interval: tuple of (start_date, end_date) in 'YYYY-MM-DD' format
    - output_dir: Directory to save the images (default: 'Po_Images')
    - resolution: Spatial resolution in meters per pixel (default: 10m)
    - max_size: Maximum image size (Sentinel Hub limit: 2500 pixels)
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Define AOI Bounding Box
    aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
    aoi_size = (min(aoi_size[0], max_size), min(aoi_size[1], max_size))  # Limit size

    # Search for available Sentinel-2 images
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=aoi_bbox,
        time=time_interval,
        fields={"include": ["id", "properties.datetime"], "exclude": []},
    )

    results = list(search_iterator)
    print(f"Total number of results: {len(results)}")

    if not results:
        print("No images found for the given AOI and time interval.")
        return []

    # Define SentinelHub evalscript for true color imagery
    evalscript_true_color = """
        //VERSION=3
        function setup() {
            return { input: [{ bands: ["B02", "B03", "B04"] }], output: { bands: 3 } };
        }
        function evaluatePixel(sample) {
            return [sample.B04, sample.B03, sample.B02];
        }
    """

    true_color_imgs = []

    # Loop through results to fetch corresponding images
    for result in results:
        timestamp = result["properties"]["datetime"]
        print(f"Fetching image for timestamp: {timestamp}")

        # Request the image
        request_true_color = SentinelHubRequest(
            evalscript=evalscript_true_color,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L2A.define_from(
                        name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
                    ),
                    time_interval=(timestamp, timestamp),
                    other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
                )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
            bbox=aoi_bbox,
            size=aoi_size,
            config=config,
        )

        # Get the image
        image_data = request_true_color.get_data()

        if image_data:
            image = image_data[0]  # Extract the first image
            true_color_imgs.append(image)

            # Save the image
            formatted_timestamp = timestamp[:19].replace(":", "-")
            filename = os.path.join(output_dir, f"sentinel_image_{formatted_timestamp}.png")
            img = Image.fromarray(np.uint8(image))
            img.save(filename)

            print(f"Saved: {filename}")

    print(f"Total images downloaded: {len(true_color_imgs)}")
    return true_color_imgs  # Return the list of images

In [6]:
dataset = xr.open_dataset("data.nc")
print(f"{dataset['n_filaments']} total filaments/ litter windrows")
list(dataset.variables.keys())

<xarray.DataArray 'n_filaments' (n_filaments: 14374)> Size: 115kB
array([    0,     1,     2, ..., 14371, 14372, 14373])
Dimensions without coordinates: n_filaments total filaments/ litter windrows


['s2_product',
 'dec_time',
 'x_centroid',
 'y_centroid',
 'lat_centroid',
 'lon_centroid',
 'n_pixels_fil',
 'limits',
 'pixel_x',
 'pixel_y',
 'pixel_spec']

In [7]:
aoi_coords_wgs84 = [12.5, 44.7, 13.0, 45.1]
time_interval = ("2024-03-09", "2024-03-10")

download_sentinel_images(aoi_coords_wgs84, time_interval)

Total number of results: 8
Fetching image for timestamp: 2024-03-10T10:08:23.784Z


KeyboardInterrupt: 

In [34]:
class DataSetLoader:
    def __init__(self, pixel_annot_path, centroids_path, chunk_size=200, patches_per_scene=4, partition_key="n_filaments"):
        self._chunk_size = chunk_size
        self._annot_path = pixel_annot_path
        self._partition_key = partition_key
        self._patches_per_scene = patches_per_scene
        self._centroids_path = centroids_path

    def _select_filaments(self, loc, time_interval):
        assert len(loc) == 4 and loc[0]<loc[2] and loc[1]<loc[3], f"Invalid coordinates {loc}"

        sp_cond = lambda d: (loc[0] <= d["Longitude"] <= loc[2]) & \
                            (loc[1] <= d["Latitude"] <= loc[3])

        tm_cond = lambda d: datetime.strptime(time_interval[0], "%Y-%m-%d") <= \
                            datetime(d['Year'], d['Month'], d['Day']) < \
                            datetime.strptime(time_interval[1], "%Y-%m-%d")

        valid = lambda x: sp_cond(x) and tm_cond(x)
        df = pd.read_excel(self._centroids_path)
        df = df[df.apply(valid, axis=1)]
        return df

    def _get_image(self, row):
        t = "-".join([row['Year'], row['Month'], row['Day']])
        time_interval = (t, t)
        image = download_sentinel_images(lat, lon, )

        download_sentinel_images(aoi_coords_wgs84, time_interval)
        aoi_coords_wgs84 = [12.5, 44.7, 13.0, 45.1]
        time_interval = ("2024-03-01", "2024-03-10")

        download_sentinel_images(aoi_coords_wgs84, time_interval)



    def load(self, locs, timeframe, data_dir="Scenes", test_size=4):

        dataset = xr.open_dataset(self._annot_path, chunks={self._partition_key: self._chunk_size})
        self._num_fils = len(np.unique(dataset["n_filaments"].values))

        centroids_df = self._select_filaments(locs, timeframe)
        for start in range(0, self._num_fils, self._chunk_size):
            sub_dataset = dataset.isel(n_filaments=slice(start, min(self._num_fils, start+self._chunk_size)))
            df = sub_dataset.to_dataframe().reset_index()

            ts = 0
            for filament_idx in df["n_filaments"].unique():
                ts += 1
                print(ts)
                if ts == test_size:
                    return None
                row = df[df["n_filaments"] == filament_idx].iloc[0]
                lat = row["lat_centroid"]
                lon = row["lon_centroid"]
                # print(row, df.columns)
                date = row["s2_product"][:20] # .str.split('T')[0]
                filament_meta = centroids_df[
                    (centroids_df["Latitude"] == lat) &\
                    (centroids_df["Longitude"] == lon)
                ]

                filament_df = df[df["n_filaments"].astype(int) == filament_idx].reset_index()
                # image = self._get_image(row)

                h = int(2*filament_df.loc[0, "y_centroid"])
                w = int(2*filament_df.loc[0, "x_centroid"])
                filament_mask = np.zeros((h, w))

                min_x = min_y = 1e8
                max_x = max_y = -1
                for idx, row in tqdm(filament_df.iterrows()):
                    if not np.isnan(row["pixel_x"]) and not np.isnan(row["pixel_y"]):
                        # print(row["pixel_x"], row["pixel_y"])
                        p_x, p_y = int(row["pixel_x"]), int(row["pixel_y"])
                        if p_x < min_x:
                            min_x = p_x
                        if p_y < min_y:
                            min_y = p_y

                        if p_x > max_x:
                            max_x = p_x
                        if p_y > max_y:
                            max_y = p_y
                        filament_mask[p_x, p_y] = 1

                h = max_x - min_x
                w = max_y - min_y
                l = 256*self._patches_per_scene
                left_pad = random.randint(0, l-w)
                right_pad = l - left_pad - w
                down_pad = random.randint(0, l-h)
                up_pad = l - down_pad - h
                mask = np.zeros((l, l))
                assert h+down_pad == l-up_pad and left_pad+w == l-right_pad, "Invalid crops"
                mask[up_pad:l-down_pad, left_pad:l-right_pad] = filament_mask[min_x:max_x, min_y:max_y]
                scene = f"S2_{date}_{lat}_{lon}"
                scene_path = os.path.join(data_dir, scene)
                os.makedirs(scene_path, exist_ok=True)
                for i in range(self._patches_per_scene):
                    patch_mask_file = f"{scene}_{i}.jpeg"
                    patch_mask_path = os.path.join(scene_path, patch_mask_file)
                    Image.fromarray(np.uint8(mask)).save(patch_mask_path)

In [35]:
loader = DataSetLoader("data.nc", "LW_centroids.xlsx", chunk_size=40)
coords = [12.5, 44.7, 13.0, 45.1]
time_interval = ("2016-03-04", "2023-03-08")
loader.load(coords, time_interval)

1


133276it [00:07, 18695.43it/s]


2


0it [00:00, ?it/s]


IndexError: index 6060 is out of bounds for axis 0 with size 2054

In [44]:
scene = "/content/Scenes/S2_b'S2A_MSIL1C_20160217T'_36.81826531261659_-6.48822047849991"
for img in os.listdir(scene):
    img = Image.open(os.path.join(scene, img))
    pos = np.array(img).sum()
    if pos:
        print(pos)
        plt.imshow(img)
        break

### Mess & Tests

In [10]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

import gc
gc.collect()

287

In [ ]:
df = pd.read_excel("LW_centroids.xlsx")
df.head()

In [ ]:
df.shape, len((df["Latitude"] + df["Longitude"]).unique())

In [ ]:
# Define AOI coordinates (min_lon, min_lat, max_lon, max_lat)
coords = [9.288940,43.012681,9.481888,43.119530]
coords = [12.5, 44.7, 13.0, 45.1]
time_interval = ("2023-03-01", "2023-03-04")  # Example time range
time_interval = ("2016-03-01", "2023-03-04")  # Example time range

# images = download_sentinel_images(aoi_coords_wgs84, time_interval)

sp_cond = lambda d, loc: (loc[0] <= d["Longitude"] <= loc[2]) & \
                    (loc[1] <= d["Latitude"] <= loc[3])
# tm_cond = lambda d, ti: datetime(ti[0].split('-')) <= datetime(d['Year'], d['Month'], d['Day']) < datetime(ti[1].split('-'))
tm_cond = lambda d: datetime.strptime(time_interval[0], "%Y-%m-%d") <= \
                    datetime(d['Year'], d['Month'], d['Day']) < \
                    datetime.strptime(time_interval[1], "%Y-%m-%d")

valid = lambda x: sp_cond(x, coords) and tm_cond(x)
df = pd.read_excel("LW_centroids.xlsx")
df = df[df.apply(valid, axis=1)]
# df = df[sp_cond(df, aoi_coords_wgs84) & tm_cond(df, time_interval)]

In [ ]:
df.shape

In [ ]:
num_filaments = len(dataset["n_filaments"])
chunk_size = 200
chunk_size = 2
dataset_path = "data.nc"
key = "n_filamnets"
patches_per_scene = 4




In [ ]:
plt.imshow(mask, cmap="gray")

In [ ]:
df["n_filaments"].unique()

In [ ]:
(filament_mask==1).sum(), min_x, min_y, max_x, max_y

In [ ]:
plt.imshow(filament_mask, cmap="gray")

In [ ]:
center_mask = filament_mask[min_x-1:max_x+1, min_y-1:max_y]
plt.imshow(center_mask, cmap="gray")

In [ ]:

key = 'n_filaments'
ds = xr.open_dataset("data.nc", chunks={key: 500})

In [ ]:
print("Dimensions:", ds.dims)
print("Sizes:", ds.sizes)

In [ ]:
for chunk in range(num_chunks)

In [ ]:
chunk_size = 500
n_time = ds.dims[key]

start = 0
# for start in range(0, n_time, chunk_size):
ds_chunk = ds.isel(n_filaments=slice(start, start + chunk_size))

df_chunk = ds_chunk.to_dataframe().reset_index()

In [ ]:
df_chunk.head()

In [ ]:
len((df_chunk["lat_centroid"] + df_chunk["lon_centroid"]).unique())

In [ ]:
df_chunk.loc[1]

In [ ]:
len(df_chunk[df_chunk["n_filaments"]==8])/13, len(df_chunk['n_bands'].unique())

In [ ]:
df_chunk.columns

In [ ]:
f1 = df_chunk[df_chunk["n_filaments"] == 0]

In [ ]:
print(len(f1))
for col in f1.columns:
    print(f"{col}: {len(f1[col].unique())}")


In [ ]:
# len((f1["pixel_x"].astype(str)+"a"+f1["pixel_y"].astype(str)).unique())
# (f1["pixel_x"].astype(str)+"a"+f1["pixel_y"].astype(str)).unique()

f1["n_bands"].unique()

In [ ]:
# mask = np.zeros(())
import numpy as np
from tqdm import tqdm
h, w = int(2*f1.loc[0, "y_centroid"]), int(2*f1.loc[0, "x_centroid"])
mask = np.zeros((h, w))

min_x = min_y = 1e8
max_x = max_y = -1
for idx, row in tqdm(f1.iterrows()):
    if not np.isnan(row["pixel_x"]) and not np.isnan(row["pixel_y"]):
        # print(row["pixel_x"], row["pixel_y"])
        p_x, p_y = int(row["pixel_x"]), int(row["pixel_y"])
        if p_x < min_x:
            min_x = p_x
        if p_y < min_y:
            min_y = p_y

        if p_x > max_x:
            max_x = p_x
        if p_y > max_y:
            max_y = p_y
        mask[p_x, p_y] = 1

In [ ]:
(mask==1).sum(), min_x, min_y, max_x, max_y

In [ ]:
center_mask = mask[min_x-1:max_x+1, min_y-1:max(min_y+200, max_y+1)]
mask.shape, center_mask.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(center_mask, cmap="gray")

In [ ]:
a = 36*44*13
len(f1)/(a)

In [ ]:
list(dataset.variables.keys())

In [ ]:
import pandas as pd

def excel_file_to_image_coords(df=None):
    if df is None:
        path = r"LM_centroids.xlsx"
        df = pd.read_excel(path)

    d_lat = 0.011575
    d_lon = 0.01339
    for idx, centroid in df.iterrows():
        lat, lon = centroid[:2]
        bbox = (lat-d_lat, lat+d_lat, lon-d_lon, lon+d_lon)
        time = ("2018-12-30T09:24:01Z")
        t = centroid.CodeT
        time = str(centroid.Date) + "-".join([t[:3], t[3:5], t[5:7]]) + "Z"
        yield {"bbox": bbox, "time": time}



sample = {
    "Latitude": 29.905255,
    "Longitude": 32.493723,
    "Dec_time": 2017.502226,
    "Str_time": "20170703T083011",
    "Year": 2017,
    "Month": 7,
    "Day": 3,
    "Date": 03/07/2017,
    "CodeT": "T083011",
    "Pixels per LW": 13,
    "Distance to land (m)": 2596,
}




import openeo

# Connect to an openEO backend (replace URL with your preferred backend)
connection = openeo.connect("https://openeo.cloud")
print("Connected to openEO backend:", connection.server_version())

bbox = [min_lon, min_lat, max_lon, max_lat]  # set appropriate coordinates
time_range = ["2021-01-01", "2021-01-31"]     # adjust to your date of interest

# Load an image collection (e.g., Sentinel-2 data)
process_graph = connection.load_collection(
    "S2L2A",
    spatial_extent={"west": bbox[0], "south": bbox[1], "east": bbox[2], "north": bbox[3]},
    temporal_extent=time_range
)